In [1]:
import pandas as pd
import numpy as np

This file will process the NBA betting odd files to determine the accuracy of winners (picked via moneyline) for each season.
Source files were downloaded from:
https://www.sportsbookreviewsonline.com/scoresoddsarchives/nba/nbaoddsarchives.htm


# Prep Results DataFrame

In [77]:
seasons = ['2007-08', 
           '2008-09',
           '2009-10',
           '2010-11',
           '2011-12',
           '2012-13',
           '2013-14',
           '2014-15',
           '2015-16',
           '2016-17',
           '2017-18',
           '2018-19',           
           '2019-20']

# helper functions

## columns

In [78]:
def get_Actual_Winner(PTS_TEAM1, PTS_TEAM2, Team_TEAM1, Team_TEAM2):
    '''
    Takes the total pts scored of both teams and returns the actual winning team
    '''
    if PTS_TEAM1 > PTS_TEAM2:
        return Team_TEAM1
    elif PTS_TEAM1 < PTS_TEAM2:
        return Team_TEAM2
    else:
        return "Error"

In [151]:
def get_Predicted_Winner(ML_TEAM1, ML_TEAM2, Team_TEAM1, Team_TEAM2):
    '''
    Takes the money line of both teams and returns the favoured team aka predicted winner.
    The more negative (smaller or "further from zero"), the more favoured the team is to win.    
    '''
    if ML_TEAM1 == "NL" and ML_TEAM2 == "NL":
        return "NL"        
    elif ML_TEAM1 < ML_TEAM2:
        return Team_TEAM1
    elif ML_TEAM2 < ML_TEAM1:
        return Team_TEAM2
    elif ML_TEAM2 == ML_TEAM1:
        return "Tie"
    else:
        return "Error"

In [83]:
def is_Predicted_Winner_Correct(Predicted_Winner, Actual_Winner):
    if Predicted_Winner == Actual_Winner:
        return True
    else:
        return False

## data processing

In [150]:
def processDataframes(df):
    ''' 
    Takes the odds dataframe and returns a dict (to be used for new row in a dataframe)
    Intent is to use in populating a separate, new dataframe that summarizes the number of correct games predicted
    '''
     # team1, every 2nd row starting at index = 0
    df_team1 = df[df.index % 2 == 0]
    df_team1.reset_index(inplace=True)
    
    # team2, every 2nd row starting at index = 1
    df_team2 = df[df.index % 2 == 1]
    df_team2.reset_index(inplace=True)

    df_vs_game = df_team1.merge(df_team2, left_index=True, right_index=True, suffixes=('_TEAM1', '_TEAM2'))
    
    df_vs_game['WINNER'] = df_vs_game.apply(lambda row: get_Actual_Winner(row.Final_TEAM1,
                                                                      row.Final_TEAM2,
                                                                      row.Team_TEAM1,
                                                                      row.Team_TEAM2), axis=1)
        
    df_vs_game['PRED_WINNER'] = df_vs_game.apply(lambda row: get_Predicted_Winner(row.ML_TEAM1,
                                                                      row.ML_TEAM2,
                                                                      row.Team_TEAM1,
                                                                      row.Team_TEAM2), axis=1)
    
    df_vs_game['IS_PRED_WINNER_CORRECT'] = df_vs_game.apply(lambda row: is_Predicted_Winner_Correct(row.PRED_WINNER,row.WINNER), axis=1)
    
    dict_results = {'SEASON': season,
                    'IS_PRED_WINNER_CORRECT':len(df_vs_game[df_vs_game['IS_PRED_WINNER_CORRECT'] == True]),
                    'TOTAL':len(df_vs_game),
                    'PCT':len(df_vs_game[df_vs_game['IS_PRED_WINNER_CORRECT'] == True]) / len(df_vs_game)}
    return dict_results

# process all files

In [162]:
column_names = ['SEASON', 'IS_PRED_WINNER_CORRECT', 'TOTAL', 'PCT']
df_result = pd.DataFrame(columns= column_names)        

In [163]:
for season in seasons:
    df = pd.read_excel('_dataVegas/nba odds ' + season + '.xlsx')   
    df_result = df_result.append(processDataframes(df), ignore_index=True)

df_result   

,SEASON,IS_PRED_WINNER_CORRECT,TOTAL,PCT
0,2007-08,908,1316,0.689970
1,2008-09,918,1315,0.698099
2,2009-10,918,1312,0.699695
3,2010-11,896,1311,0.683448
4,2011-12,744,1074,0.692737
5,2012-13,901,1314,0.685693
6,2013-14,892,1319,0.676270
7,2014-15,913,1311,0.696415
8,2015-16,912,1316,0.693009
9,2016-17,869,1309,0.663866


In [164]:
# total accuracy, across all seasons
x = np.sum(df_result.IS_PRED_WINNER_CORRECT)
y = np.sum(df_result.TOTAL)
print('Vegas predict winner (moneyline) across all seasons (2007-08 to 2019-20): ', x/y, 'Total GAmes:', y)

Vegas predict winner (moneyline) across all seasons (2007-08 to 2019-20):  0.683505526189332 Total GAmes: 16648


# 2019-20 pre bubble (in order to align with kaggle data set)

In [165]:
df = pd.read_excel('_dataVegas/nba odds ' + season + '.xlsx')

In [166]:
# looking for the row where the pre bubble games exist (to match the kaggle dataset)
df.loc[(df.index<1805) & ((df.index>1795))]

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H
1796,301,573,V,LALakers,29,34,30,29,122,2.5,235.5,115,116
1797,301,574,H,NewOrleans,33,28,34,19,114,233.5,2,-135,1.5
1798,301,575,V,Washington,35,28,31,30,124,5.5,7,-310,2.5
1799,301,576,H,GoldenState,28,29,19,34,110,230,233.5,250,115
1800,302,577,V,Houston,31,32,28,32,123,8.5,10,-600,11.5
1801,302,578,H,NewYork,36,37,28,24,125,230.5,231,425,113.5
1802,302,579,V,Portland,35,34,23,38,130,223.5,220,225,108.5
1803,302,580,H,Orlando,28,30,30,19,107,7,6.5,-275,6.5
1804,302,581,V,Utah,31,28,35,32,126,8.5,10.5,-650,4


In [167]:
df_2019_pre_bubble = df.loc[df.index<1800]
df_2019_pre_bubble

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H
0,1022,501,V,NewOrleans,30,31,25,31,122,231.5,229.5,230,113
1,1022,502,H,Toronto,27,29,32,29,130,6.5,6.5,-280,6
2,1022,503,V,LALakers,25,29,31,17,102,227,3.5,-180,5
3,1022,504,H,LAClippers,22,40,23,27,112,1.5,224,150,110.5
4,1023,505,V,Detroit,27,27,29,36,119,210,211,240,104
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,301,572,H,Denver,40,33,27,33,133,4.5,2.5,-140,pk
1796,301,573,V,LALakers,29,34,30,29,122,2.5,235.5,115,116
1797,301,574,H,NewOrleans,33,28,34,19,114,233.5,2,-135,1.5
1798,301,575,V,Washington,35,28,31,30,124,5.5,7,-310,2.5


In [168]:
df_2019_pre_covid_results = pd.DataFrame(columns =column_names)

In [169]:
df_2019_pre_covid_results = df_2019_pre_covid_results.append(processDataframes(df_2019_pre_bubble), ignore_index=True)

In [170]:
df_2019_pre_covid_results

,SEASON,IS_PRED_WINNER_CORRECT,TOTAL,PCT
0,2019-20,603,900,0.67


# * Ignore - Scrap Notes
# =========================

for season in seasons:
    df = pd.read_excel('_dataVegas/nba odds ' + season + '.xlsx')
    
    # team1, every 2nd row starting at index = 0
    df_team1 = df[df.index % 2 == 0]
    df_team1.reset_index(inplace=True)
    
    # team2, every 2nd row starting at index = 1
    df_team2 = df[df.index % 2 == 1]
    df_team2.reset_index(inplace=True)

    df_vs_game = df_team1.merge(df_team2, left_index=True, right_index=True, suffixes=('_TEAM1', '_TEAM2'))
    
    df_vs_game['WINNER'] = df_vs_game.apply(lambda row: get_Actual_Winner(row.Final_TEAM1,
                                                                      row.Final_TEAM2,
                                                                      row.Team_TEAM1,
                                                                      row.Team_TEAM2), axis=1)
        
    df_vs_game['PRED_WINNER'] = df_vs_game.apply(lambda row: get_Predicted_Winner(row.ML_TEAM1,
                                                                      row.ML_TEAM2,
                                                                      row.Team_TEAM1,
                                                                      row.Team_TEAM2), axis=1)
    
    df_vs_game['IS_PRED_WINNER_CORRECT'] = df_vs_game.apply(lambda row: is_Predicted_Winner_Correct(row.PRED_WINNER,row.WINNER), axis=1)
    
    df_result = df_result.append({'SEASON': season,
                             'IS_PRED_WINNER_CORRECT':len(df_vs_game[df_vs_game['IS_PRED_WINNER_CORRECT'] == True]),
                             'TOTAL':len(df_vs_game),
                             'PCT':len(df_vs_game[df_vs_game['IS_PRED_WINNER_CORRECT'] == True]) / len(df_vs_game)}, ignore_index=True)
    
    
    